# Item 35: Avoid Causing State Transitions in Generators with `throw`

Another advanced generator feature (apart from `yield from` and `send`) is the `throw` method for re-raising `Exception` instances within generator functions. When the `throw` method is called, the next occurrance of a `yield` expression re-raises the provided `Exception` instance after its output is received instead of continuing normally.

In [3]:
# Example of the throw in a generator
class MyError(Exception):
    pass

def my_generator():
    yield 1
    yield 2
    yield 3

it = my_generator()
print(next(it)) # Yield 1
print(next(it)) # Yield 2
print(it.throw(MyError('test error')))

1
2


MyError: test error

In [4]:
# When we call throw, the generator function may catch the injected exception, with a standard try/except 
# compound statement that surrounds the last yield expression that was executed
def my_generator():
    yield 1

    try:
        yield 2
    except MyError:
        print('Got MyError!')
    else:
        yield 3

    yield 4

it = my_generator()
print(next(it)) # Yield 1
print(next(it)) # Yield 2
print(it.throw(MyError('test error')))

1
2
Got MyError!
4


This functionality provides a two-way communication channel between a generator and its caller that can be useful in certain situatons.

In [6]:
# Here we implement a generator that reilies on the throw method. Whenever the Reset exception is raised by the
# yield expression, the counter resets itself to its original period
class Reset(Exception):
    pass

def timer(period):
    current = period
    while current:
        current -= 1
        try:
            yield current
        except Reset:
            current = period

In [9]:
# We can connect this caounter reset event to an external input that's polled every second
def check_for_reset():
    # Poll for external event
    pass

def announce(remaining):
    print(f'{remaining} ticks remaining')

def run():
    it = timer(4)
    while True:
        try:
            if check_for_reset():
                current = it.throw(Reset())
            else:
                current = next(it)
        except StopIteration:
            break
        else:
            announce(current)

run()

3 ticks remaining
2 ticks remaining
1 ticks remaining
0 ticks remaining


The above code works but its hard to read and the level of nesting makes the code noisy. 

A simpler approach to implementing this functionality is to define a stateful closure using an iterable container object.

In [10]:
# Here, we redifine the timer generator by using such a class
class Timer:
    def __init__(self, period):
        self.current = period
        self.period = period
    
    def reset(self):
        self.current = self.period

    def __iter__(self):
        while self.current:
            self.current -= 1
            yield self.current

In [11]:
# Now the run method can do much simpler iteration by using a for statement. The code is much easier to follow
def run():
    timer = Timer(4)
    for current in timer:
        if check_for_reset():
            timer.reset()
        announce(current)

run()

3 ticks remaining
2 ticks remaining
1 ticks remaining
0 ticks remaining


The author suggests that we avoid using `throw` entirely and instead use an iterable class if we need this type of exceptional behavior.